Als erstes wollen wir das Trainingsdatenset einlesen.

In [41]:
import pandas as pd
original_data = pd.read_csv("train.csv")
print(type(original_data))

<class 'pandas.core.frame.DataFrame'>


Nun soll dieses Trainingsdatenset noch mal aufgeteilt werden in ein Validierungsdatenset (20%) und in ein eigentliches Trainigsdatenset (80%)

In [42]:
from sklearn.model_selection import train_test_split

training_set, validation_set = train_test_split(original_data, test_size=0.2)
print(len(training_set))
print(len(validation_set))

712
179


Nun sollen wir eine Regel für das Überleben alleine aufgrund der Klasse des Tickets (Pclass) definieren und auf unserem neuen Trainingsset definieren

In [43]:
classes_passengers = {1: 0, 2:0, 3:0}
classes_survived = {1: 0, 2:0, 3:0}

for passenger in training_set.itertuples(index=False):
    classes_survived[passenger[2]] += passenger[1]
    classes_passengers[passenger[2]] += 1

print(classes_passengers)
print(classes_survived)

class_survival_rate = {1: 0, 2:0, 3:0}
for i in range(1,4):
    class_survival_rate[i] = classes_survived[i] / classes_passengers[i]

print(class_survival_rate)

{1: 173, 2: 143, 3: 396}
{1: 110, 2: 70, 3: 94}
{1: 0.6358381502890174, 2: 0.48951048951048953, 3: 0.23737373737373738}


Anwenden der Überlebensregel auf unser Validierungsset

In [45]:
import random
val_predictions = []
for passenger in validation_set.itertuples(index=False):
    predicted_survival = 1 if random.random() < class_survival_rate[passenger[2]] else 0
    val_predictions.append(predicted_survival)

correct_predictions = validation_set['Survived'] == val_predictions
print(f"korrekte Vorhersagen: {sum(correct_predictions)}")
accuracy = sum(correct_predictions) / len(correct_predictions)
print("Genauigkeit auf den Validierungsdaten:", accuracy)

korrekte Vorhersagen: 103
Genauigkeit auf den Validierungsdaten: 0.5754189944134078


Nun sollten wir dieselbe Funktion über die Testdaten von Kaggle laufen lassen

In [56]:
from pathlib import Path

original_test_set = pd.read_csv("test.csv")

val_predictions = []
for passenger in original_test_set.itertuples():
    predicted_survival = 1 if random.random() < class_survival_rate[passenger[2]] else 0
    val_predictions.append(predicted_survival)


output = pd.DataFrame({'PassengerId': original_test_set["PassengerId"], 'Survived': val_predictions})
output.to_csv('my_submission.csv', index=False)



![alt text](img.png)

 #### Logistische Regression mit Pclass

Trainieren Sie eine logistische Regression mit den Variablen 'Pclass'. Verwenden Sie die Klasse sklearn.linear_model.LogisticRegression. Berechnen Sie die Accuracy auf dem Validierungsset.

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logReg = LogisticRegression()
logReg.fit(training_set[["Pclass"]], training_set[["Survived"]])

prediction = logReg.predict(validation_set[["Pclass"]])

accuracy = accuracy_score(validation_set[["Survived"]], prediction)
print("Genauigkeit auf dem Validierungsset:", accuracy)

print(logReg.score(validation_set[["Pclass"]], validation_set[["Survived"]]))

0.6703910614525139
Genauigkeit auf dem Validierungsset: 0.6703910614525139


C:\Users\haase\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Dieser Code füllt alle leeren einträge in der Altersspalte mit dem Mittelwert der Alterswerte aus dem Trainingsset.

In [63]:
validation_set["Age"].fillna(training_set["Age"].median(skipna=True), inplace=True)
training_set["Age"].fillna(training_set["Age"].median(skipna=True), inplace=True)

print(training_set)

     PassengerId  Survived  Pclass   
467          468         0       1  \
538          539         0       3   
73            74         0       3   
543          544         1       2   
849          850         1       1   
..           ...       ...     ...   
489          490         1       3   
802          803         1       1   
46            47         0       3   
408          409         0       3   
147          148         0       3   

                                             Name     Sex   Age  SibSp  Parch   
467                    Smart, Mr. John Montgomery    male  56.0      0      0  \
538                      Risien, Mr. Samuel Beard    male  28.0      0      0   
73                    Chronopoulos, Mr. Apostolos    male  26.0      1      0   
543                             Beane, Mr. Edward    male  32.0      1      0   
849  Goldenberg, Mrs. Samuel L (Edwiga Grabowska)  female  28.0      1      0   
..                                            ...     ...